In [13]:
###########################################################################
# IMPORTS
###########################################################################

import pandas as pd
import agentpy as ap
import random
import math

###############################################################################
# WORKER AGENT
###############################################################################

class Worker(ap.Agent):

    ###########################################################################
    # INITIALIZATION
    ###########################################################################

    def setup(self):

        # Initialize attributes:
        self.mu = random.uniform(0, 0.01)
        self.sigma = random.uniform(0, math.sqrt(2 * self.mu))
        self.wealth = 1
        self.employer = None
        self.employer_id = None
        self.employer_size = None
        self.wage = 0
        self.output = 0
        self.network = None
        self.search = False
        self.switched = False

    ###########################################################################
    # SETTERS
    ###########################################################################
    
    def update_employer(self, firm):
        self.employer=firm
        self.employer_id=firm.id

    def update_wealth(self, wealth):
        self.wealth = wealth
        self.wage = wealth

    def init_network(self):
        self.network = (
            self.model.workers
            .select(self.model.workers.id != self.id)
            .random(n=self.model.p['num_neighbors'], replace=False)
            .to_list()
        )
    
    ###########################################################################
    # WEALTH FUNCTIONS
    ###########################################################################
    
    def produce(self):
        
        # Increment wealth following multiplicative wealth dynamic.
        W = random.normalvariate(mu=0, sigma=1)
        output = (
            self.wealth * math.exp(
                (self.mu - (self.sigma ** 2) / 2) + self.sigma * W
            )
        )

        # Update output.
        self.output = output
    
        return output
    
    def calc_time_avg_growth_rate(self, firm, include_self=False):

        # Get list of workers from firms.
        workers = firm.workers
    
        # If a list of workers are passed, calculate the growth rate of
        # wealth for the group as a cooperating unit.
        if include_self:
            workers = workers + [self]

        # Number of workers in the list.
        N = len(workers)

        # Contribution of each worker in the list.
        g_i = [
            (worker.mu - (worker.sigma ** 2) / (2 * N)) for worker in workers
        ]

        # Growth rate based on total contributions of workers.
        g = sum(g_i) / N

        return g

    ###########################################################################
    # FIRM SELECTION
    ###########################################################################

    def select_empty_firms(self):

        # Choose a firm at random from entire population of firms with zero
        # workers.
        empty_firms = (
            self.model.firms
            .select(self.model.firms.size == 0)
            .random()
            .to_list()
        )

        return empty_firms
    
    def select_neighbor_firms(self):

        # Create an agent list of the employers from the agent's network.
        firms = ap.AgentDList(
            self.model,
            (
                self.network
                .select(self.network.employer.id != self.employer.id)
                .employer
            )
        )

        # 1% of the agents select firms the general population.
        if random.random() < 0.01:

            workers = (
                self.model.workers
                .select(self.model.workers.employer.id != self.employer.id)
            )

            if len(workers) >= self.model.p['num_neighbors']:
                workers = (
                    workers
                    .random(n=self.model.p['num_neighbors'], replace=False)
                    .to_list()
                )

            firms = ap.AgentDList(
                self.model,
                workers.employer
            )
        
        return firms

    def rank_firms(self):
    
        # Start-ups are firms with zero employees.
        startup = self.select_empty_firms()

        # Neighbor's firms are from a random sample of an agent's network.
        neighbor_firms = self.select_neighbor_firms()

        # Potential firms: startups plus neighbor's firms.
        potential_firms = neighbor_firms + startup

        # Calculate the time average growth rate of each firm.
        g_f = [
            self.calc_time_avg_growth_rate(firm=firm, include_self=True)
            for firm in potential_firms
        ]
    
        # Get the index value of the firm with the highest growth rate.
        max_idx = g_f.index(max(g_f))
 
        return potential_firms[max_idx], g_f[max_idx]
    
    def firm_selection(self):

        self.search = True

        curr_g = self.calc_time_avg_growth_rate(firm=self.employer)
        new_firm, new_g = self.rank_firms()

        if new_g > curr_g:

            self.switched = True
            self.employer.separate(self)
            new_firm.hire(self)
            self.update_employer(firm=new_firm)

###############################################################################
# FIRM AGENT
###############################################################################

class Firm(ap.Agent):
    
    def setup(self):

        # Initialization
        self.workers = []
        self.size = 0
        self.output = 0
        self.output_per_worker = 0
        self.num_hired = 0
        self.num_separated = 0
    
    ###########################################################################
    # HIRING AND SEPARATIONS
    ###########################################################################

    def hire(self, worker):

        self.workers.append(worker)
        self.size = len(self.workers)
        self.num_hired += 1

    def separate(self, worker):

        self.workers.remove(worker)
        self.size = len(self.workers)
        self.num_separated += 1
    
    ###########################################################################
    # PRODUCTION AND DISTRIBUTION
    ###########################################################################
    
    def produce(self):

        if not self.size:
            self.output = 0
        else:
            # Total output of the firm, which is the sum of all individual
            # contributions
            output = sum([worker.produce() for worker in self.workers])

            # Set output.
            self.output = output
    
    def distribute(self):

        if not self.size:
            self.output_per_worker = 0
        else:
            self.output_per_worker = (self.output / self.size)

            for worker in self.workers:
                worker.update_wealth(wealth=self.output_per_worker)

###############################################################################
# MODEL CLASS
###############################################################################

class Market(ap.Model):

    def setup(self):
        """
        Defines the model's actions before the first simulation step.
        Can be overwritten to initiate agents and environments.
        """

        # Create worker and firm agents.
        self.workers = ap.AgentList(self, self.p['n_workers'], Worker)
        self.firms = ap.AgentList(self, 2*self.p['n_workers']+1, Firm)

        # Initialize worker networks.
        self.workers.init_network()

        # Initialize workers into singleton firms.
        for i in range(self.p['n_workers']):
            worker = self.workers[i]
            firm = self.firms[i]

            # Assign the worker to the firm, update the firm's worker list.
            worker.update_employer(firm=firm)
            firm.hire(worker)
        
        # Update worker employer size.
        self.workers.employer_size = self.workers.employer.size

    def step(self):
        """
        Defines the model's actions during each simulation step (excluding t==0).
        Can be overwritten to define the models main dynamics.
        """

        # Randomly select workers to look for new jobs.
        (
            self.workers
            .random(
                n=math.ceil(self.model.p['n_workers']*self.model.p['active'])
            )
            .to_list()
            .firm_selection()
        )
        
        # Update agent employer size.
        self.workers.employer_size = self.workers.employer.size

        # Produce and distribute output.
        self.firms.produce()
        self.firms.distribute()

    def update(self):
        """
        Defines the model's actions after each simulation step (including t==0).
        Can be overwritten for the recording of dynamic variables.
        """

        # Collect microdata.
        self.workers.record(
            ['wealth','wage','output','employer_id','employer_size','search','switched']
        )
        self.firms.record(
            ['size','output','output_per_worker','num_hired','num_separated']
        )

        # Reset hiring and separation numbers.
        self.firms.num_hired = 0
        self.firms.num_separated = 0

        # Reset worker job search values.
        self.workers.search = False
        self.workers.switched = False

    def end(self):
        """
        Defines the model's actions after the last simulation step.
        Can be overwritten for final calculations and reporting.
        """
    
        # Record initial attributes for workers in separate dataset and add it
        # to the model output when the simulation is complete.
        # self.model.output['initial_attributes'] = ap.DataDict({'Worker': pd.DataFrame(data=data)})
        data = [
            {'id': worker.id, 'mu': worker.mu, 'sigma': worker.sigma, 'wealth': worker.wealth}
            for worker in self.workers
        ]
        self.model.output['initial_attributes'] = pd.DataFrame(data=data)

# def run_and_save_model(parameters):
#     model = Market(parameters)
#     results = model.run()
#     results.save(exp_name='ee_results', exp_id=1, path='results')

### Execute Model

In [ ]:
parameters = {
    'seed': 92,
    'steps': 10,
    'n_workers': 10,
    'num_neighbors': 4,
    'active': 0.10
 }

model = Market(parameters)
results = model.run()
# results.save(exp_name='ee_results', exp_id=1, path='results')